In [ ]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)\s+batch_size\s+(?P<batch_size>[\d.]+)"


  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)
  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
      'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
      'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
      'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_walltime', "batch_size"
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'h200_batch'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
ncore = str(1)
!OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)
df

,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_walltime,batch_size
0,43,6,1,41,20,2,12.888506,0.00106,0.001046,0.002105,13.295584,1.463584,14.759168,1551.537189,1
1,43,6,1,41,20,2,10.023621,0.00106,0.001046,0.002105,5.934944,1.380512,191.298721,193.437662,32
2,43,6,1,41,20,2,10.296694,0.00106,0.001046,0.002105,4.801248,1.065920,308.345764,310.785448,64
3,43,6,1,41,20,2,10.116276,0.00106,0.001046,0.002105,4.292825,1.049600,550.531250,552.837427,128
4,43,6,1,41,20,2,10.364089,0.00106,0.001046,0.002105,4.306448,1.057056,1103.507690,1105.517208,256
5,43,6,1,41,20,2,10.128363,0.00106,0.001046,0.002105,4.303428,1.069056,2204.424316,2206.387023,512
6,43,6,1,41,20,2,10.435541,0.00106,0.001046,0.002105,4.307534,0.992256,4411.907227,4414.233626,1024
7,43,6,1,41,20,2,11.088426,0.00106,0.001046,0.002105,4.309789,0.974848,8827.422852,8828.834415,2048
8,43,6,1,41,20,2,10.360324,0.00106,0.001046,0.002105,4.313686,0.999296,17669.857422,17671.681560,4096
9,43,6,1,41,20,2,11.300924,0.00106,0.001046,0.002105,4.350989,1.056768,35644.359375,35646.936662,8192


In [4]:
!nproc
!lscpu

192


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   192
  On-line CPU(s) list:    0-191
Vendor ID:                GenuineIntel
  Model name:             INTEL(R) XEON(R) PLATINUM 8568Y+
    CPU family:           6
    Model:                207
    Thread(s) per core:   2
    Core(s) per socket:   48
    Socket(s):            2
    Stepping:             2
    CPU max MHz:          4000.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4600.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid aperfmperf tsc_known_freq 

In [5]:
!nvidia-smi

Wed May 21 08:08:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:4C:00.0 Off |                    0 |
| N/A   28C    P0             76W /  700W |       0MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----